<a href="https://colab.research.google.com/github/foxtrotmike/CS909/blob/master/mnist_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Transformers for MNIST: A foundational tutorial
By [Fayyaz Minhas](https://sites.google.com/view/fayyaz/home)

Transformers have revolutionized the field of natural language processing (NLP) and beyond, offering significant improvements in tasks like text translation, summarization, and generation. At the heart of this architecture are several innovative components, each contributing to the transformer's ability to handle sequential data effectively, without the limitations of recurrence or convolution. This tutorial will introduce you to these key components, aiming to provide a solid understanding of how transformers work.

## Multiple Multi-headed Attention Blocks
The transformer model employs multiple blocks of multi-headed attention mechanisms. These are designed to enable the model to focus on different parts of the input sequence simultaneously, allowing it to capture a wide range of dependencies between words or features. Each "head" in these attention blocks can be thought of as an independent feature extraction mechanism, looking at the input from a different perspective. By having multiple heads, the model can aggregate diverse insights about the relationships within the data.

## Layer Normalization
Layer normalization is a technique used within transformers to stabilize the neural network's learning process. It is applied to the outputs of each sub-layer (such as attention or feed-forward layers) within the transformer blocks. By normalizing the activations across the features for each data sample in a mini-batch, layer normalization helps in speeding up training and improving the overall performance of the model.

## Standardization Across Features of the Same Input
This process, often implemented as part of layer normalization, involves adjusting the features of the input data to have a mean of zero and a standard deviation of one. Standardization ensures that all features contribute equally to the model's learning, preventing any one feature with larger numerical values from dominating the learning process.

## Skip Connections
Also known as residual connections, skip connections are a critical component of the transformer architecture. They allow the output of one layer to be added to the output of a later layer, helping to mitigate the vanishing gradient problem in deep neural networks. This promotes the flow of gradients through the network, making it easier to train deeper models.

## Various Types of Positional Encodings
Transformers do not inherently process sequence order, unlike recurrent neural networks. To address this, positional encodings are added to the input embeddings to provide the model with information about the order of the sequence elements. These encodings can be fixed (based on sine and cosine functions of different frequencies) or learnable parameters. This feature enables transformers to effectively capture the sequential nature of the input data.

## “Class” Tokens
In tasks such as sequence classification, a special “class” token is often prepended to the input sequence. The final state corresponding to this token is used as the aggregate representation of the entire sequence for classification purposes. This approach allows the model to use global sequence information effectively for making predictions.


## Masking Strategies
Masking is a crucial strategy in transformer models, particularly for tasks such as sentence completion or machine translation. It involves selectively hiding parts of the input or future tokens from the model during training. This forces the model to predict missing words or the next word in a sequence without relying on the actual next word, fostering a more robust understanding of context and relationships within the data. However, for vision transformer implementation below, we will be ignoring masking.

By understanding these components and their interplay within the transformer architecture, you'll be equipped to dive deeper into the model's applications and nuances. Each element plays a pivotal role in the transformer's ability to learn from sequential data, making it a powerful tool in the machine learning practitioner's toolkit.

# Coding

 we will explore the implementation and workings of a Vision Transformer (ViT) for image classification tasks. The Vision Transformer applies principles of the transformer architecture, originally designed for natural language processing, to the domain of computer vision. Let's break down the code into its components and understand how it transforms an image into a format suitable for the transformer, processes it through multiple attention blocks, and finally uses it for classification.

## Image Patching and Positional Encoding
The first step in the Vision Transformer is to convert an image into a sequence of flattened patches. This is akin to tokenizing a sentence in NLP tasks. The ViT class starts by ensuring the image dimensions are divisible by the patch size. Each image is then rearranged into non-overlapping patches using the rearrange function from einops. These patches are flattened and projected to a higher-dimensional space (the embedding dimension of the transformer) using a linear layer (self.patch_to_embedding). This process effectively treats each patch as a "word" in the context of NLP.

Next, positional encodings are added to the patch embeddings to retain information about the original location of each patch within the image. The Vision Transformer also introduces a learnable "class token" at the beginning of the sequence. This class token is a unique embedding that is updated through the transformer layers and used for classification. The combination of patch embeddings, positional embeddings, and the class token forms the input to the transformer.

## Multi-headed Attention and Transformer Blocks
Within the Transformer class, the input sequence passes through multiple layers, each consisting of a multi-head attention block followed by a feed-forward neural network. Both of these components are encapsulated in residual connections (Residual) and preceded by layer normalization (PreNorm), following the standard transformer architecture.

### Attention Mechanism:
The multi-headed attention mechanism (Attention class) allows the model to focus on different parts of the image simultaneously. It does this by computing scaled dot-product attention across different "heads," enabling the model to capture various aspects of the input.

### Feed-Forward Networks:
After the attention block, the data passes through a feed-forward network (FeedForward class) in each transformer layer. This network applies a simple sequence of linear transformations and non-linear activations to further process the information.

## Classification
After the input sequence has been processed by the transformer layers, the updated class token is extracted and passed through a final classifier (the mlp_head in the ViT class). This classifier consists of linear layers and GELU activations, culminating in a prediction over the defined number of classes.

## Summary of the Approach
* Image Patching:
 The input image is divided into patches, which are then flattened and projected to a higher dimension.
* Positional Encoding:
Positional embeddings are added to the patch embeddings to give the model information about the original position of each patch in the image. A learnable class token is also prepended to the sequence.
* Transformer Processing:
The sequence of embeddings is processed through several layers of multi-headed attention and feed-forward networks, all within residual connections and preceded by normalization.
* Classification:
The class token, now enriched with information from across the image, is used to make a final classification.

This coding walkthrough illustrates the innovative way the Vision Transformer adapts the transformer architecture from text to images, treating image patches as analogous to words in a sentence. It highlights the transformer's versatility and its powerful capability for both NLP and computer vision tasks.

In [ ]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 1.2 MB/s eta 0:00:00


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import torch.nn.functional as F
from einops import rearrange

class Residual(nn.Module):
    # Implements a residual connection followed by a function fn.
    def __init__(self, fn):
        super().__init__()
        self.fn = fn  # The function to apply before adding the input back.

    def forward(self, x, **kwargs):
        # Applies the function and adds the input x back to the result.
        return self.fn(x, **kwargs) + x

class PreNorm(nn.Module):
    # Applies layer normalization before a function fn.
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)  # Normalizes the input across the specified dimension.
        self.fn = fn  # The function to apply after normalization.

    def forward(self, x, **kwargs):
        # Normalizes the input and then applies the function.
        return self.fn(self.norm(x), **kwargs)

class FeedForward(nn.Module):
    # Implements a feedforward neural network with one hidden layer.
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),  # Linear transformation to the hidden dimension.
            nn.GELU(),  # GELU non-linearity.
            nn.Linear(hidden_dim, dim)  # Linear transformation back to the original dimension.
        )

    def forward(self, x):
        # Passes the input through the feedforward network.
        return self.net(x)

class Attention(nn.Module):
    # Implements the multi-head self-attention mechanism.
    def __init__(self, dim, heads=8):
        super().__init__()
        self.heads = heads  # Number of attention heads.
        self.scale = dim ** -0.5  # Scaling factor for the dot products.

        # Linear layer to project the input into queries, keys, and values.
        self.to_qkv = nn.Linear(dim, dim * 3, bias=False)
        # Linear layer to project the concatenated outputs back to the original dimension.
        self.to_out = nn.Linear(dim, dim)

    def forward(self, x, mask=None):
        b, n, _, h = *x.shape, self.heads
        # Projects input to queries, keys, and values.
        qkv = self.to_qkv(x)
        # Rearranges the projections for multi-head attention processing.
        q, k, v = rearrange(qkv, 'b n (qkv h d) -> qkv b h n d', qkv=3, h=h)

        # Calculates the dot products of queries and keys, scales the result.
        dots = torch.einsum('bhid,bhjd->bhij', q, k) * self.scale

        if mask is not None:
            # Applies masking if provided, for handling variable sequence lengths.
            mask = F.pad(mask.flatten(1), (1, 0), value=True)
            assert mask.shape[-1] == dots.shape[-1], 'mask has incorrect dimensions'
            mask = mask[:, None, :] * mask[:, :, None]
            dots.masked_fill_(~mask, float('-inf'))
            del mask

        # Applies softmax to get the attention weights.
        attn = dots.softmax(dim=-1)

        # Uses einsum to compute weighted sum of values.
        out = torch.einsum('bhij,bhjd->bhid', attn, v)
        # Rearranges the output back to the original input shape.
        out = rearrange(out, 'b h n d -> b n (h d)')
        # Projects the output back to the original dimension.
        out = self.to_out(out)
        return out

class Transformer(nn.Module):
    # Implements a sequence of transformer blocks.
    def __init__(self, dim, depth, heads, mlp_dim):
        super().__init__()
        self.layers = nn.ModuleList([])
        for _ in range(depth):
            # Each block consists of a multi-head attention layer and a feedforward layer, both wrapped in residual and normalization layers.
            self.layers.append(nn.ModuleList([
                Residual(PreNorm(dim, Attention(dim, heads=heads))),
                Residual(PreNorm(dim, FeedForward(dim, mlp_dim)))
            ]))

    def forward(self, x, mask=None):
        # Processes the input through each block in sequence.
        for attn, ff in self.layers:
            x = attn(x, mask=mask)  # Applies attention.
            x = ff(x)  # Applies feedforward network.
        return x

class ViT(nn.Module):
    # Vision Transformer (ViT) for image classification.
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, channels=3):
        super().__init__()
        assert image_size % patch_size == 0, 'image dimensions must be divisible by the patch size'
        num_patches = (image_size // patch_size) ** 2
        patch_dim = channels * patch_size ** 2  # Dimensionality of the flattened patch.

        self.patch_size = patch_size

        # Learnable position embeddings.
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        # Linear layer to project flattened patches to the dimensionality of the transformer.
        self.patch_to_embedding = nn.Linear(patch_dim, dim)
        # Learnable class token.
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        # The transformer core.
        self.transformer = Transformer(dim, depth, heads, mlp_dim)

        self.to_cls_token = nn.Identity()

        # The "head" of the ViT to make final class predictions.
        self.mlp_head = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.GELU(),
            nn.Linear(mlp_dim, num_classes)
        )

    def forward(self, img, mask=None):
        p = self.patch_size

        # Rearranges the input image into non-overlapping patches.
        x = rearrange(img, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1=p, p2=p)

        # Projects patches to embeddings.
        x = self.patch_to_embedding(x)

        # Prepends the class token to the sequence of embedded patches.
        cls_tokens = self.cls_token.expand(img.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        # Adds position embeddings to the patch embeddings.
        x += self.pos_embedding
        # Processes the sequence through the transformer.
        x = self.transformer(x, mask)

        # Extracts the class token and applies the classification head.
        x = self.to_cls_token(x[:, 0])
        return self.mlp_head(x)


In [ ]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
model = ViT(image_size=28, patch_size=7, num_classes=10, channels=1,
            dim=64, depth=6, heads=8, mlp_dim=128)
# Define a transform to normalize the data
transform = transforms.Compose([
    transforms.Resize((28, 28)),  # Ensure the image size is 28x28
    transforms.ToTensor(),  # Convert images to tensors
    transforms.Normalize((0.5,), (0.5,))  # Normalize the images
])

# Download and load the training data
trainset = MNIST('', download=True, train=True, transform=transform)
trainloader = DataLoader(trainset, batch_size=64, shuffle=True)

# Download and load the test data
testset = MNIST('', download=True, train=False, transform=transform)
testloader = DataLoader(testset, batch_size=64, shuffle=False)
import torch.optim as optim

# Define the loss function and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.003)

epochs = 5
for epoch in range(epochs):
    running_loss = 0
    for images, labels in tqdm(trainloader):
        # Reset the gradients to zero
        optimizer.zero_grad()

        # Forward pass
        output = model(images)
        loss = loss_function(output, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    else:
        print(f"Training loss: {running_loss/len(trainloader)}")

correct = 0
total = 0
with torch.no_grad():
    model.eval()
    for images, labels in tqdm(testloader):
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy of the network on the 10000 test images: {100 * correct / total} %')


  0%|          | 0/938 [00:00<?, ?it/s]

Training loss: 0.34086699017694894


  0%|          | 0/938 [00:00<?, ?it/s]

Training loss: 0.13769662428869686


  0%|          | 0/938 [00:00<?, ?it/s]

...

Training loss: 0.0998129521939419


  0%|          | 0/938 [00:00<?, ?it/s]

Training loss: 0.08331751398621068


  0%|          | 0/157 [00:00<?, ?it/s]

Accuracy of the network on the 10000 test images: 97.64 %
